In [5]:
import os, glob, re
import pandas as pd
from tqdm import tqdm

# ==== ĐƯỜNG DẪN ====
data_dir  = "00_brca"
# meta_path = "gdc_sample_sheet.2025-10-21.tsv"
meta_path = "breast_cancer_annotation.csv"

# ==== ĐỌC METADATA ====
# meta = pd.read_csv(meta_path, sep="\t")
# meta = meta[["File ID", "File Name", "Case ID", "Tissue Type"]].copy()
# meta["File Name"] = meta["File Name"].fillna("").astype(str).str.strip()
meta = pd.read_csv(meta_path)
meta.rename(columns={"primary_diagnosis": "Tissue Type"}, inplace=True)
meta["File Name"] = meta["File Name"].fillna("").astype(str).str.strip()

# ==== QUÉT FILE ====
all_paths = glob.glob(os.path.join(data_dir, "**", "*.tsv"), recursive=True)
all_paths = [p.replace("\\", "/") for p in all_paths]

path_index = {os.path.basename(p).lower(): p for p in all_paths}

# ==== GỘP THÀNH MA TRẬN (GỘP THEO gene_id) ====
gene_expr_list = []
kept_samples = []

for _, row in tqdm(meta.iterrows(), total=len(meta)):
    file_name = row["File Name"].strip().replace("\\", "/")
    key = os.path.basename(file_name).lower()
    path = path_index.get(key)
    if path is None:
        matches = [p for name, p in path_index.items() if key in name]
        if len(matches) == 1:
            path = matches[0]
        else:
            print(f"[⚠️] Không tìm thấy file cho {file_name}")
            continue

    df = pd.read_csv(path, sep="\t", comment="#")

    # Ưu tiên TPM → FPKM-UQ → FPKM
    value_col = "tpm_unstranded"
    if value_col not in df.columns:
        raise ValueError(f"Mẫu {os.path.basename(path)} không có cột {value_col}. Dừng để đảm bảo đơn vị đồng nhất.")


    # Ép số, bỏ NA, gộp theo gene_id (nếu 1 gene xuất hiện nhiều dòng)
    tmp = df[["gene_id", value_col]].copy()
    tmp = tmp.dropna(subset=["gene_id", value_col])
    tmp[value_col] = pd.to_numeric(tmp[value_col], errors="coerce")
    tmp = tmp.dropna(subset=[value_col])

    s = tmp.groupby("gene_id", as_index=True)[value_col].sum()
    s.name = os.path.basename(file_name)
    gene_expr_list.append(s)
    kept_samples.append(s.name)

if not gene_expr_list:
    raise RuntimeError("Không gộp được mẫu nào. Kiểm tra lại dữ liệu/metadata.")

# Ma trận (genes × samples) -> transpose để được (samples × gene_id)
expr_matrix = pd.concat(gene_expr_list, axis=1).fillna(0.0)  # index = gene_id, columns = samples
expr_matrix = expr_matrix.loc[~expr_matrix.index.duplicated(keep="first")]
expr_matrix = expr_matrix.T  # ==> samples × gene_id

print(f"✅ Ma trận (samples × gene_id): {expr_matrix.shape}")

# ==== THÊM LABEL (tuỳ chọn, bạn có thể bỏ nếu chưa cần) ====
#meta["label"] = meta["Tissue Type"].apply(lambda x: "Tumor" if "tumor" in str(x).lower() else "Normal")
meta["label"] = meta["Tissue Type"].apply(
    lambda x: "Infiltrating duct carcinoma" 
    if "infiltrating duct carcinoma" in str(x).lower() 
    else "Lobular carcinoma"
)
expr_matrix.index = expr_matrix.index.astype(str).str.strip()

expr_with_label = expr_matrix.merge(
    meta[["File Name","label"]],
    how="left",
    left_index=True,
    right_on="File Name"
)
cols = ["File Name","label"] + [c for c in expr_with_label.columns if c not in ["File Name","label"]]
expr_with_label = expr_with_label[cols]

# ==== LƯU FILE (TSV) ====
out_path_matrix = "brca_matrix_subtyping.tsv"                 # samples × gene_name, không có label
out_path_withlab = "brca_matrix_with_label_subtyping.tsv"     # thêm 2 cột đầu: File Name, label

expr_matrix.to_csv(out_path_matrix, sep="\t")
expr_with_label.to_csv(out_path_withlab, sep="\t", index=False)

print(f"💾 Đã lưu: {out_path_matrix}")
print(f"💾 Đã lưu: {out_path_withlab}")


100%|██████████| 955/955 [01:36<00:00,  9.91it/s]


✅ Ma trận (samples × gene_id): (955, 60660)
💾 Đã lưu: brca_matrix_subtyping.tsv
💾 Đã lưu: brca_matrix_with_label_subtyping.tsv


In [6]:
df =pd.read_csv("brca_matrix_subtyping.tsv", sep="\t")


In [7]:
df_shape = pd.read_csv("brca_matrix_with_label_subtyping.tsv", sep="\t")
df_shape

,File Name,label,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,22c2b380-799e-4fad-ae38-46a916c592d5.rna_seq.a...,Infiltrating duct carcinoma,15.3427,0.2774,113.9613,18.5449,5.5132,8.0859,13.0406,49.6999,...,0.0,0.0000,0.1724,0.0,0.0000,0.0000,10.4667,0.0,0.0255,0.2924
1,fd6f4c9b-ee43-4939-8cfa-2e447aedbcf3.rna_seq.a...,Infiltrating duct carcinoma,27.2879,0.0335,305.2314,16.3078,9.7574,5.9120,20.6773,514.9600,...,0.0,0.8249,0.1997,0.0,0.8113,0.0000,11.8319,0.0,0.0566,0.6186
2,a02fb212-a6cc-40b2-9d31-481fc1ce0911.rna_seq.a...,Infiltrating duct carcinoma,63.3753,0.4280,103.9729,31.0735,10.9337,5.6034,33.1148,28.8386,...,0.0,0.0000,0.5819,0.0,0.0000,0.0000,15.2066,0.0,0.0876,0.4700
3,de01516e-43f0-4f96-8ac6-ab543a314829.rna_seq.a...,Infiltrating duct carcinoma,29.6290,2.6355,53.9820,36.5087,17.4623,3.2205,25.3966,14.8870,...,0.0,19.4496,1.5358,0.0,51.0154,0.0000,23.8949,0.0,0.0607,0.8104
4,e6bad6ec-c178-4684-99e8-2504781a022b.rna_seq.a...,Infiltrating duct carcinoma,36.0147,0.6135,67.7105,20.0504,6.5149,1.5262,28.4660,14.5685,...,0.0,1.5093,0.0794,0.0,0.0000,0.0000,5.2369,0.0,0.0377,0.0809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,d6450e34-ebf4-4749-a27e-8e3139960afb.rna_seq.a...,Infiltrating duct carcinoma,23.9018,0.8334,62.8190,5.5016,1.6630,6.9363,23.3521,42.0305,...,0.0,0.0000,0.3237,0.0,0.0000,0.0441,7.9739,0.0,0.0365,1.9350
951,94367ad2-de29-4beb-8dab-f79174894cd2.rna_seq.a...,Infiltrating duct carcinoma,14.6263,0.0000,96.3039,6.0784,1.8824,6.8059,4.5557,82.8362,...,0.0,1.1283,0.4158,0.0,0.0000,0.0000,8.2422,0.0,0.0422,2.6596
952,081b4937-85e7-44b3-9124-a612f4f2e0ca.rna_seq.a...,Infiltrating duct carcinoma,33.5312,0.3349,88.4685,25.6249,4.4532,15.6406,11.8806,74.3154,...,0.0,0.0000,0.5590,0.0,0.0000,0.0000,9.3017,0.0,0.0914,3.0734
953,5e36c4a2-fc6b-418f-80ab-dbd95b26a39e.rna_seq.a...,Lobular carcinoma,56.4105,0.9309,97.2697,12.1995,4.8589,6.5231,51.1202,83.6936,...,0.0,0.0000,0.1741,0.0,0.0000,0.0000,8.1532,0.0,0.0079,0.4544


In [4]:
df

,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,2e64abe2-6024-4d28-9e09-560ce2a9fd15.rna_seq.a...,18.6332,5.4450,85.1730,8.6113,3.6796,11.9609,22.9706,56.2461,8.8515,...,0.0,0.0,0.2115,0.0,0.0000,0.0000,8.0427,0.0,0.0000,0.5581
1,96a45a70-d4b5-4090-b603-73b7379679d6.rna_seq.a...,43.8845,0.1846,100.4090,25.4762,6.6869,12.5896,32.1927,60.7617,13.3435,...,0.0,0.0,0.3945,0.0,1.1168,0.0000,21.4515,0.0,0.0921,1.2571
2,1d468785-141d-40ca-acb1-d6d85a8c9d7b.rna_seq.a...,14.2779,0.3068,94.0155,4.7815,3.3801,8.2730,35.4841,87.7875,9.1559,...,0.0,0.0,0.4238,0.0,0.0000,0.0000,9.5242,0.0,0.0366,0.8387
3,748e4eaa-2b96-4dce-a903-c7df733d7f50.rna_seq.a...,83.3794,6.7515,111.2359,13.0596,7.6468,13.9962,31.6161,60.0317,24.8455,...,0.0,0.0,0.2836,0.0,0.0000,0.0000,7.6388,0.0,0.0350,0.5210
4,e614fbb4-7574-4704-9525-c0aea4c10fc6.rna_seq.a...,39.1278,0.0850,71.2884,13.3303,4.9899,10.4031,51.1152,39.2914,11.6945,...,0.0,0.0,0.1981,0.0,0.0000,0.0000,19.8870,0.0,0.1043,0.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,f8cfabf6-dc25-4141-be6c-817dd439a74b.rna_seq.a...,53.5208,25.4515,64.2265,8.7310,1.9823,5.2649,14.3132,28.8747,21.6000,...,0.0,0.0,0.1143,0.0,0.0000,0.0136,15.8023,0.0,0.0903,0.3554
1227,a37efe02-9662-4b90-8251-003ccb96aed4.rna_seq.a...,11.9798,0.2934,142.4774,7.2553,4.0132,15.3207,17.0466,33.4487,4.7144,...,0.0,0.0,0.3672,0.0,0.0000,0.0000,13.6184,0.0,0.0975,6.0138
1228,02bbe1b1-44ff-426b-a40e-47a01bc4cf7d.rna_seq.a...,38.6251,5.4839,101.2572,9.9249,9.8658,40.2793,20.0921,47.0518,13.1625,...,0.0,0.0,0.3626,0.0,0.0000,0.0000,4.7971,0.0,0.1492,1.0251
1229,4331397f-eaf9-4e43-8fa2-e30ad3643aca.rna_seq.a...,23.9626,0.0000,117.6828,27.8003,8.8753,8.7305,23.2782,77.3552,11.9860,...,0.0,0.0,0.2515,0.0,0.0000,0.0000,25.0655,0.0,0.0199,0.6825
